### Week four exercise

This project simulates generation and interpretation of **telemetry data** (as would be sent via **MQTT** or a similar protocol to a cloud service) from a sensor. The LLM generates the data and a natural language summary of the data. Results are displayed in a Gradio UI.

In [ ]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess
from IPython.display import Markdown, display
from huggingface_hub import login

In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
hf_token = os.getenv('HF_TOKEN')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
login(hf_token, add_to_git_credential=True)

In [ ]:
ollama_url = "http://localhost:11434/v1"
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

openai = OpenAI(api_key=openai_api_key)

In [ ]:
models = [
    "gpt-4o-mini",
    "gpt-4o",
    "deepseek-r1:1.5b",
    "llama3.2",
]

clients = {
    "gpt-4o-mini": openai,
    "gpt-4o": openai,
    "deepseek-r1:1.5b": ollama,
    "llama3.2": ollama,
}


def get_client(model_name: str):
    """Return the API client to use for the given model (Ollama or OpenAI)."""
    client = clients.get(model_name)
    if client is ollama:
        return ollama
    return openai

In [ ]:
TELEMETRY_USER_PROMPT = """
Simulate telemetry from an IoT sensor as it would be sent via MQTT to a cloud service.

1. Generate a realistic JSON object with plausible sensor measurements (e.g. temperature, humidity, pressure, battery, timestamp, device_id).
2. Then write a brief natural-language summary interpreting the data, noting any trends or anomalies.

Format your response exactly as follows:
TELEMETRY:
<valid JSON object here>
SUMMARY:
<your brief summary here>
"""

In [ ]:
def generate_telemetry(model_name: str) -> tuple[str, str]:
    """Call the LLM to generate sensor telemetry (MQTT-style) and a natural-language summary."""
    client = get_client(model_name)
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": TELEMETRY_USER_PROMPT}],
        temperature=0.7,
    )
    text = response.choices[0].message.content or ""
    telemetry_text, summary_text = "", ""
    if "SUMMARY:" in text:
        parts = text.split("SUMMARY:", 1)
        telemetry_text = parts[0].replace("TELEMETRY:", "").strip()
        summary_text = parts[1].strip()
    else:
        telemetry_text = text
    return telemetry_text, summary_text

In [ ]:
with gr.Blocks(theme=gr.themes.Soft(), title="Sensor Telemetry Simulator") as ui:
    gr.Markdown("## Sensor Telemetry Simulator\nGenerate and interpret IoT sensor telemetry as would be sent via MQTT to a cloud service.")
    with gr.Row():
        model = gr.Dropdown(choices=models, value="gpt-4o-mini" if "gpt-4o-mini" in models else models[0], label="Model")
        generate_btn = gr.Button("Generate telemetry", variant="primary")
    with gr.Row():
        telemetry_out = gr.TextArea(label="Telemetry data (JSON)", lines=12, placeholder="Generated sensor payload will appear here…")
        summary_out = gr.TextArea(label="Summary", lines=12, placeholder="Natural-language interpretation will appear here…")
    generate_btn.click(fn=generate_telemetry, inputs=[model], outputs=[telemetry_out, summary_out])

ui.launch(inbrowser=True)
